# Groene boekje
* Use `dbutils.add_lexicon`!
* Use `bulk_add_anahashes` and `connect_anahases_to_wordforms`

INL takse about 3 minutes (without anahashes)

In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import ticclat.dbutils
import pandas as pd
import numpy as np

## To do

* Add relationships between models (should make processing an xml file faster)
* Use sessions better: https://docs.sqlalchemy.org/en/latest/orm/session_basics.html#when-do-i-construct-a-session-when-do-i-commit-it-and-when-do-i-close-it
* Add multiple documents
* Extract vocabulary

In [76]:
# Read information to connect to the database and put it in environment variables
import os
with open('ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()

In [77]:
db_name = 'ticclat'
os.environ['dbname'] = db_name

In [78]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

engine = create_engine("mysql://{}:{}@localhost/{}".format(os.environ['user'], 
                                                           os.environ['password'], 
                                                           os.environ['dbname']))
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url))

Session = sessionmaker(bind=engine)

True


In [79]:
# from ticclat.lexicon_schema import AnalyzedWordform, Document, Lemmata, TokenAttestation, Wordform, Base
import ticclat.ticclat_schema as schema

In [80]:
# create tables
schema.Base.metadata.create_all(engine)

In [81]:
from sqlalchemy import inspect

inspector = inspect(engine)

In [82]:
# Get table information
print(inspector.get_table_names())

['anahashes', 'corpora', 'corpusId_x_documentId', 'documents', 'lexica', 'lexical_source_wordform', 'source_x_wordform_link', 'text_attestations', 'wordform_links', 'wordforms']


# Load Groene Boekje data into Pandas

In [83]:
GB_basepath = "/Users/pbos/projects/ticclat/data/GB/"

In [84]:
GB1914_path = GB_basepath + "1914/22722-8.txt"
GB1995_path = GB_basepath + "1995-2005/1995/GB95_002.csv"
GB2005_path = GB_basepath + "1995-2005/2005/GB05_002.csv"

In [85]:
df_GB1995 = pd.read_csv(GB1995_path, sep=';', names=["word", "syllables", "see also", "disambiguation",
                                                     "grammatical tag", "article",
                                                     "plural/past/attrib", "plural/past/attrib syllables",
                                                     "diminu/compara/past plural", "diminu/compara/past plural syllables",
                                                     "past perfect/superla", "past perfect/superla syllables"],
                        encoding='utf8') # encoding necessary for later loading into sqlalchemy!

In [13]:
# df_GB1995
# df_GB1995['see also'].dropna().map(lambda x: x[:8]).unique()

There's a lot of stuff in there... clean up time.

Multiple columns:
- Some entries have "@" in them. They seem to be general rules for some kinds of words. Not sure what to do with these, so will just filter out for now.

First column:
- Has entries with multiple values (e.g. "Zwols, Zwoller")
- Sometimes has multiple rows with the same word, probably in different meanings or something, e.g. "aal1", "aal2" and "aal3".
    + In this case, the fourth column has a note to disambiguate the meanings.

Second column: Splits words according to syllables, we can disregard this.

Third: a "See also \[other word\]" note (they all start with "Zie ook")

Fourth: disambiguation of a duplicate word (see first column)

Fifth: grammatical tag, i.e. noun, adjective, verb, etc.

Sixth: the proper definite article for the noun ("de" or "het").

Seventh: first inflection form. Sometimes left empty.
- If a noun: plural form
- If a verb: past tense singular
- If an adjective (or pronoun (`vnw.`)): attributive form

Nineth: second inflection form. Often left empty, probably for regular or easy forms.
- If a noun: diminutive
- If an adjective: comparative
- If a verb: past tense plural

Eleventh:
- If a verb: past perfect (voltooid verleden tijd?)
- If an adjective: superlative

8, 10, 12: Syllables of the preceding.

## Clean up

### Wordforms
- 1 needs most work:
  + Split up if it has a comma and two (or more) words --> Make it two (or more) rows; check that the other columns also have **the same number** of comma separated words and match those, otherwise just duplicate other columns.
  + Make a new column for rows with a "duplicate entry number" postfixed; put the number there, remove it from the first row.
    - Note that column 3 also uses the postfix number to refer to specific duplicates!
- 2, 8, 10 and 12 can go, we have no need for pronunciation information currently.
- We have no place in the current schema for 5, so we drop it as well for now.
- Column 3 is for links only.
- Columns 7, 9 and 11 also contain wordforms, so we give these their own rows as well. For now, we don't need links, so we just extract the words and worry about efficient linking later.

### Links
This lexicon also contains a lot of word links.

- Obviously, the three inflection columns, 7 9 and 11, give clear immediate links between wordforms, i.e. grammatical ones.
- Column 3 also gives a link, of words with similar meanings, i.e. a semantic link.

### Grammatical function
Column 5 may at some point be used to add grammatical information, if we ever plan on using this.

## Actual wordforms

In [80]:
df_GB1995_wordforms = df_GB1995.drop(["syllables", "see also", "disambiguation", "grammatical tag", "article",
                                      "plural/past/attrib syllables", "diminu/compara/past plural syllables", "past perfect/superla syllables"], axis=1)

In [81]:
df_GB1995_wordforms.sample(10)

,word,plural/past/attrib,diminu/compara/past plural,past perfect/superla
24334,endocrinoloog,NaN,NaN,NaN
64178,openingsbeeld,openingsbeelden,NaN,NaN
2405,afrekening,afrekeningen,NaN,NaN
22143,driekleur,NaN,NaN,NaN
96519,venkelwater,NaN,NaN,NaN
102361,voorwas,NaN,NaN,NaN
90090,temporeel,temporele,NaN,NaN
96691,verbintenis,verbintenissen,NaN,NaN
52527,maatverdeling,maatverdelingen,NaN,NaN
35614,"heroi@\sering""",NaN,NaN,NaN


## Categories

In [86]:
df_GB1995["grammatical tag"].unique()

array(['ongespecif.', 'znw.', 'bnw.', 'ww.min_zich', 'uitdr.', 'bijw.',
       'voorz.', 'voegw.', 'ww.met_zich', 'eigenn.', 'telw.', 'vnw.',
       'de', nan, 'lidw.', 'het', 'tw.'], dtype=object)

In [101]:
df_GB1995[df_GB1995["grammatical tag"] == 'tw.']

,word,syllables,see also,disambiguation,grammatical tag,article,plural/past/attrib,plural/past/attrib syllables,diminu/compara/past plural,diminu/compara/past plural syllables,past perfect/superla,past perfect/superla syllables
32039,goh,goh,NaN,NaN,tw.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41036,jemig,je/mig,NaN,NaN,tw.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83036,soit,soit,NaN,NaN,tw.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We should probably skip `uitdr.` (sayings). Maybe also `eigenn.` (proper names). Will leave them in for now.

# Wordform clean up

## Mysterious "@" rows
Let's take out the weird "@" rows first to see what we can do with them.

In [15]:
def nd_any(*args):
    result = (args[0] | args[0])
    for arg in args:
        result = (result | arg)
    return result

In [23]:
df_GB1995_at = df_GB1995[nd_any(*tuple(df_GB1995[col].str.contains('@') for col in df_GB1995.columns))]

In [25]:
df_GB1995_at.sample(4)

,word,syllables,see also,disambiguation,grammatical tag,article,plural/past/attrib,plural/past/attrib syllables,diminu/compara/past plural,diminu/compara/past plural syllables,past perfect/superla,past perfect/superla syllables
42104,kampeerauto,kam/peer/au/to,NaN,NaN,znw.,de[m.],kampeerauto@@s,kam/peer/au/to@@s,NaN,NaN,NaN,NaN
24520,enque@^tering,en/que@^/te/ring,NaN,NaN,znw.,de[v.],NaN,NaN,NaN,NaN,NaN,NaN
15822,cinema,ci/ne/ma,zie ook kinema,NaN,znw.,de[m.],cinema@@s,ci/ne/ma@@s,NaN,NaN,NaN,NaN
14722,buurtcomite@',buurt/co/mi/te@',NaN,NaN,znw.,het,buurtcomite@'s,buurt/co/mi/te@'s,NaN,NaN,NaN,NaN


In [26]:
x = df_GB1995_at.values.flatten()
x = pd.Series(x).dropna().to_frame()
x = x[x[0].str.contains('@')]

In [27]:
ats = x[0].str.extract('(\@[^\w]*)')

In [28]:
ats[0].unique()

array(['@`', '@@', "@'", '@\\', '@`/', '@\\/', '@` ', "@'/", '@\\";',
       '@\\@@', '@+', '@^', '@^/', "@'-", "@' ", '@', '@@ ', '@=', '@`-',
       '@\\-'], dtype=object)

Ahhh, they seem to be diacritic markers!
- @\` is accent grave on the previous character
- @\' is accent aigu on the previous character
- @@ seems to be apostrophe

But then... a whole zoo of quite rare ones. Let's check out examples.

In [29]:
for at in ats[0].unique():
    print(x[ats[0].str.contains(at.replace('\\', "\\\\"))].head())

                              0
0                           a@`
1                           a@`
204       achtervolgingssce@`ne
205  ach/ter/vol/gings/sce@`/ne
468            afscheidssce@`ne
                      0
18                 a@@s
19                 a@@s
20               a@@tje
21               a@@tje
42  aankoopprogramma@@s
                              0
24         aanbevelingscomite@'
25   aan/be/ve/lings/co/mi/te@'
240               actiecomite@'
241           ac/tie/co/mi/te@'
246              actiecomite@'s
                    0
78       aardolie@\n"
79     aard/oli/e@\n"
96      abiturie@\nt"
97   abi/tu/ri/e@\nt"
102   abiturie@\nten"
                               0
205   ach/ter/vol/gings/sce@`/ne
469          af/scheids/sce@`/ne
1009                  am/pe@`/re
1015                 am/pe@`/res
1021           am/pe@`/re/me/ter
                      0
283  ac/tu/a/ri/e@\/le"
301      ade/no/i@\/de"
307     ade/no/i@\/den"
325       ae@\/ro/club"
331      ae@\/ro/clubs

Ok, so the three-character ones seem to be wrong: / are all just syllable separators, with a - behind them are just koppeltekens, with space is just space, etc., so let's cut it down to a two-character filter (actually also just one, there's a naked @ as well... though is that one actually naked or does the following character there also have special meaning? **make sure**):

In [30]:
ats = x[0].str.extract('(\@[^\w]?)')

In [31]:
ats[0].unique()

array(['@`', '@@', "@'", '@\\', '@+', '@^', '@', '@='], dtype=object)

In [32]:
for at in ats[0].unique():
    print(at)
    print(x[ats[0].str.contains(at.replace('\\', "\\\\").replace('+', "\+").replace('^', "\^"))].head())

@`
                              0
0                           a@`
1                           a@`
204       achtervolgingssce@`ne
205  ach/ter/vol/gings/sce@`/ne
468            afscheidssce@`ne
@@
                      0
18                 a@@s
19                 a@@s
20               a@@tje
21               a@@tje
42  aankoopprogramma@@s
@'
                              0
24         aanbevelingscomite@'
25   aan/be/ve/lings/co/mi/te@'
240               actiecomite@'
241           ac/tie/co/mi/te@'
246              actiecomite@'s
@\
                    0
78       aardolie@\n"
79     aard/oli/e@\n"
96      abiturie@\nt"
97   abi/tu/ri/e@\nt"
102   abiturie@\nten"
@+
                 0
1296      aperc@+u
1297     aper/c@+u
1302   aperc@+u@@s
1303  aper/c@+u@@s
2328  bagagerec@+u
@^
                          0
3816       bliksemenque@^te
3817   blik/sem/en/que@^/te
7284    cou@^te que cou@^te
7285  cou@^/te que cou@^/te
7416                cre@^pe
@
         0
0      a@`
1      a@`
18   

- @\` is accent grave on the previous character
- @\' is accent aigu
- @\\ is trema
- @+ is cedilla
- @^ is accent circumflex
- @= is a tilde
- @@ is apostrophe between the characters

The naked @ requires a bit more work:

In [33]:
naked_at = x[~nd_any(*tuple(ats[0].str.contains(at.replace('\\', "\\\\").replace('+', "\+").replace('^', "\^")) for at in ats[0].unique() if at != '@'))]

In [34]:
naked_at[~naked_at[0].str.contains('/')]

,0
5892,CO@2-emissie
5898,CO@2-emissies
5904,CO@2-heffing
5916,CO@2-probleem
18618,"kadeee@n, kadees"
18702,kakofoniee@n
18726,kalligrafiee@n
18774,kankertherapiee@n
18930,"kerkprovincies, kerkprovincie@n"
18942,kernideee@n


Ok, so the presumed naked @ is actually two possible things I wrongly filtered out:
- @2 is 2 in subscript, as in CO$_2$ (and only that)
- @n comes exclusively before an e that should have a trema on it; the backslash that's normally used is probably omitted to avoid confusion with newline-character \\n

We will normalize the diacritics below when we have gathered all wordforms in one array, for easier processing.

## Gather wordforms

Now to gather all actual wordforms, i.e. those in columns 1, 7, 9 and 11, also splitting by comma.

In [35]:
wordform_df = pd.concat((df_GB1995["word"], df_GB1995[df_GB1995.columns[6]], df_GB1995[df_GB1995.columns[8]], df_GB1995[df_GB1995.columns[10]]))\
                .dropna()

In [36]:
wordform_df[wordform_df.str.contains(', ')].head()

100      aandachtstreep, aandachtsstreep
104    aandachttrekker, aandachtstrekker
747                   aanvraag, aanvrage
767        aan weerszijden, aan weerszij
997                   Aarlenaar, Arelaar
dtype: object

In [37]:
wordform_df[wordform_df.str.contains(':')].sample(10)

94097        UFO@@tje:
66631          paatje:
23720        EHBO@@er:
15228       caviaatje:
14885     cameliaatje:
23151        echootje:
21969       dramaatje:
16307       collietje:
90331    teringzootje:
14953      canapeetje:
dtype: object

ffs, some words have colons in them, what do those mean then?

In [38]:
df_GB1995_colon = df_GB1995[nd_any(*tuple(df_GB1995[col].str.contains(':') for col in df_GB1995.columns))]

In [39]:
df_GB1995_colon.head()

,word,syllables,see also,disambiguation,grammatical tag,article,plural/past/attrib,plural/past/attrib syllables,diminu/compara/past plural,diminu/compara/past plural syllables,past perfect/superla,past perfect/superla syllables
1051,abc,abc,NaN,NaN,znw.,het,abc@@s,abc@@s,abc@@tje:,abc/tje,NaN,NaN
1163,acacia,aca/cia,NaN,NaN,znw.,de[m.],acacia@@s,aca/cia@@s,acaciaatje:,aca/ci/a/tje,NaN,NaN
1248,accu,ac/cu,NaN,NaN,znw.,de[m.],accu@@s,ac/cu@@s,accuutje:,ac/cu/tje,NaN,NaN
2981,agenda,agen/da,NaN,NaN,znw.,de,agenda@@s,agen/da@@s,agendaatje:,agen/da/tje,NaN,NaN
3751,amfora,am/fo/ra,NaN,NaN,znw.,de,amfora@@s,am/fo/ra@@s,amforaatje:,am/fo/ra/tje,NaN,NaN


I have no idea what this means, so will just filter it out.

Also, there are "words" that are actually several words... arrggh!

In [40]:
has_comma = wordform_df.str.contains(', ')
wordform_df = pd.concat((wordform_df[~has_comma],) + tuple(pd.Series(row.split(', ')) for row in wordform_df[has_comma]))

In [41]:
wordform_df[wordform_df.str.contains(', ')].head()

Series([], dtype: object)

### Filter out crap
- Colons behind some words: we remove the colons and leave the rest of the word in.
- Words that are in fact several words, like "aan weerszijden" or "schoof vooruit". Remove those entries, split the words and append them to the end.
- Strip whitespace from either end of words (some apparently have it).
- Remove "footnote" numbers postfixed to duplicate words: like with colons.
- Retain only unique words after the above procedures.

In [42]:
# draw some samples to check for remaining weird shit
wordform_df.sample(10)

102905             vrijdagse
9100          beroepsrenners
105005           bliezen weg
42577     karaktertekeningen
47865              kurkenzak
29831           gemotiveerde
89622       tegenwoordigheid
52920                 mammie
92704        getranscendeerd
78648            schaduwvlek
dtype: object

#### Remove colons

In [43]:
wordform_df = wordform_df.str.replace(':', '')

In [44]:
wordform_df[wordform_df.str.contains(':')].head()

Series([], dtype: object)

#### Split multiple word entries

In [45]:
# surround space with any-character, because some single words also just have space padding
multi_word = wordform_df.str.contains('. .', regex=True)
wordform_df = pd.concat((wordform_df[~multi_word],) + tuple(pd.Series(row.split(' ')) for row in wordform_df[multi_word]))

In [46]:
wordform_df[wordform_df.str.contains('. .')]

Series([], dtype: object)

#### Strip whitespace
Ok, so it's actually just one word, but still.

In [47]:
wordform_df[wordform_df.str.contains(' ')]

0    ganzenvederen 
dtype: object

In [48]:
wordform_df = wordform_df.str.strip()

In [49]:
wordform_df[wordform_df.str.contains(' ')]

Series([], dtype: object)

#### Remove duplicate word footnote numbers

In [50]:
duplicates = wordform_df.str.contains('[0-9]$', regex=True)
wordform_df = pd.concat((wordform_df[~duplicates], wordform_df[duplicates].str.replace('[0-9]$', '', regex=True)))

In [51]:
wordform_df.tail(), wordform_df[wordform_df.str.contains('[0-9]$', regex=True)]

(0    schrikpoeder
 0       tegengift
 0       voldoende
 0     zweetpoeder
 0     zweetpoeder
 dtype: object, Series([], dtype: object))

#### Retain only unique wordforms

In [52]:
wordform_df = pd.Series(wordform_df.unique())

In [53]:
wordform_df.sort_values().head()

190783            (etc.)
190750          (kwader)
101204          (werken)
190719               @@s
77526     @@s-Gravenhage
dtype: object

#### Extra: remove stuff between parentheses
This turned up when sorting. We should remove "etc.", which is a abbreviation, but we could keep the others.

In [54]:
wordform_df = wordform_df.sort_values().str.strip("()")

In [55]:
wordform_df.head()

190783              etc.
190750            kwader
101204            werken
190719               @@s
77526     @@s-Gravenhage
dtype: object

#### Extra 2: abbreviations
Are there any more of them?

In [56]:
wordform_df[wordform_df.str.contains('.', regex=False)]

190783                  etc.
5677                      B.
50019                     M.
6750                     bc.
20974                    dr.
21536                   drs.
21537             drs.-titel
21745                    ds.
37653                   ing.
38928                    ir.
48211                   lic.
54726                    mr.
68916                  prof.
82795       st.-jakobsschelp
156117    st.-jakobsschelpen
191083              vademen.
dtype: object

Ok, we remove the ones that are "pure" abbreviations, i.e. that end in a period.

In [57]:
abbreviation = wordform_df.str.contains('\.$')
wordform_df = wordform_df[~abbreviation]

In [58]:
wordform_df[wordform_df.str.contains('.', regex=False)]

21537             drs.-titel
82795       st.-jakobsschelp
156117    st.-jakobsschelpen
dtype: object

#### Extra 3: Retain only unique wordforms... again

In [59]:
wordform_df = pd.Series(wordform_df.unique())

In [60]:
wordform_df.sort_values().head()

2                  @@s
3       @@s-Gravenhage
4    @@s-Hertogenbosch
5                  @@t
6             A-biljet
dtype: object

Ok, this is a problem. "'s" is not a separate word, it really belongs to some other word, that we split it off from.

### Aaaand again

After discussion, we decided to keep in the multi-word wordforms after all. We will see how to deal with them in TICCL later.

For the occasion, let's also just put everything in one function.

In [16]:
wordform_df = pd.concat((df_GB1995["word"],
                         df_GB1995["plural/past/attrib"],
                         df_GB1995["diminu/compara/past plural"],
                         df_GB1995["past perfect/superla"]))\
                .dropna()
has_comma = wordform_df.str.contains(', ')
wordform_df = pd.concat((wordform_df[~has_comma],) + tuple(pd.Series(row.split(', ')) for row in wordform_df[has_comma]))

In [17]:
def clean_wordform_df(wordform_df):
    # remove colons
    wordform_df = wordform_df.str.replace(':', '')
    # strip whitespace
    wordform_df = wordform_df.str.strip()
    # remove duplicate word footnote numbers
    duplicates = wordform_df.str.contains('[0-9]$', regex=True)
    wordform_df = pd.concat((wordform_df[~duplicates], wordform_df[duplicates].str.replace('[0-9]$', '', regex=True)))
    # remove parentheses around some words
    wordform_df = wordform_df.sort_values().str.strip("()")
    # remove abbreviations
    abbreviation = wordform_df.str.contains('\.$')
    wordform_df = wordform_df[~abbreviation]
    # remove duplicates
    wordform_df = pd.Series(wordform_df.unique())
    return wordform_df

In [18]:
wordform_clean_df = clean_wordform_df(wordform_df)

In [19]:
def check_cleanliness_wordform_df(wordform_df, head=5):
    print("Random sample:")
    display(wordform_df.sample(10))
    print("Colons, periods:")
    display(wordform_df[wordform_df.str.contains(':')].head(head))
    display(wordform_df[wordform_df.str.contains('.', regex=False)].head(head))
    print("White space padding:")
    display(wordform_df[wordform_df.str.contains('^ | $')].head(head))
    print("Trailing numbers:")
    display(wordform_df[wordform_df.str.contains('[0-9]$', regex=True)].head(head))
    print("Parentheses:")
    display(wordform_df[wordform_df.str.contains('\(|\)', regex=True)].head(head))
    print("Abbreviations:")
    display(wordform_df[wordform_df.str.contains('\.$')].head(head))
    
    print("Finally, just the first entries of sorted df:")
    display(wordform_df.sort_values().head(head))
    display(wordform_df.sort_values().tail(head))    

In [20]:
check_cleanliness_wordform_df(wordform_df)

Random sample:


58126             natiewagens
5544                 autogiro
47255              sloot krom
101994    voorlichtingsbudget
35218               helpertje
42493        kapitein-vlieger
21581          verhuurde door
62513        onmenselijkheden
64831               opruimers
9465               bestelling
dtype: object

Colons, periods:


4321         AOW@@er:
15233        CDA@@er:
15664    chocolaatje:
16259    collegaatje:
23720       EHBO@@er:
dtype: object

5876             B.
7000            bc.
21845           dr.
22433          drs.
22434    drs.-titel
dtype: object

...

Abbreviations:


5876       B.
7000      bc.
21845     dr.
22433    drs.
22649     ds.
dtype: object

Finally, just the first entries of sorted df:


107697             (werken)
105814             (werken)
78215      @@s anderendaags
81091        @@s-Gravenhage
81109     @@s-Hertogenbosch
dtype: object

1                          zwoord
1                        zwoorden
2974        zworen af zweerden af
95084     zworen uit zweerden uit
110578            zworen zweerden
dtype: object

In [21]:
check_cleanliness_wordform_df(wordform_clean_df)

Random sample:


71594          imperfectum
18104      bestuurskundige
170805            vanboven
84637     koopmanstraditie
68571          hoofdmannen
150246          spieringen
72108             indiaans
138332           rouwkoets
22236          blueszanger
23861              borduur
dtype: object

Colons, periods:


Series([], dtype: object)

40318                 drs.-titel
151892          st.-jakobsschelp
151893        st.-jakobsschelpen
161098    ten tweede (etc.) male
170166           vademen. vadems
dtype: object

...

Abbreviations:


Series([], dtype: object)

Finally, just the first entries of sorted df:


1              @@s anderendaags
2                @@s-Gravenhage
3             @@s-Hertogenbosch
4    @@t is dief en diefjesmaat
5                      A-biljet
dtype: object

196929                     zwoord
196930                   zwoorden
196931      zworen af zweerden af
196932    zworen uit zweerden uit
196933            zworen zweerden
dtype: object

So, there's some remaining issues here that should be at least duly noted (though ideally dealt with):

- Some words have `(etc.)` in it, like `ten tweede (etc.) male`. This should be expanded into `ten derde male`, and so forth for all counting words.
- Other parenthesized words like `op de(n) duur` should be considered as two words, `op de duur` and `op den duur`.
- I'm not sure what the period in `vademen. vadems` means, though I suspect the period is a mistyped comma.

**We will leave these things as they are for now.**

One more pressing remaining issue is that there are apparently "multiple words" that are not separated by comma at all! See `zworen af zweerde af` which should be two terms. Let's check what this looks like in the original table:

In [22]:
df_GB1995[df_GB1995['word'].str.contains('zweren', na=False)]

,word,syllables,see also,disambiguation,grammatical tag,article,plural/past/attrib,plural/past/attrib syllables,diminu/compara/past plural,diminu/compara/past plural syllables,past perfect/superla,past perfect/superla syllables
2974,afzweren1,af/zwe/ren,NaN,(door verzwering afvallen),ww.min_zich,NaN,zwoor af zweerde af,zwoor af zweer/de af,zworen af zweerden af,zwo/ren af zweer/den af,afgezworen,af/ge/zwo/ren
2975,afzweren2,af/zwe/ren,NaN,(onder ede verwerpen),ww.min_zich,NaN,zwoer af,zwoer af,NaN,NaN,afgezworen,af/ge/zwo/ren
10467,bezweren,be/zwe/ren,NaN,NaN,ww.min_zich,NaN,bezwoer,be/zwoer,NaN,NaN,bezworen,be/zwo/ren
40629,inzweren,in/zwe/ren,NaN,NaN,ww.min_zich,NaN,NaN,NaN,NaN,NaN,ingezworen,in/ge/zwo/ren
78181,samenzweren,sa/men/zwe/ren,NaN,NaN,ww.min_zich,NaN,zwoer samen,zwoer sa/men,NaN,NaN,samengezworen,sa/men/ge/zwo/ren
95084,uitzweren,uit/zwe/ren,NaN,NaN,ww.min_zich,NaN,zwoor uit zweerde uit,zwoor uit zweer/de uit,zworen uit zweerden uit,zwo/ren uit zweer/den uit,uitgezworen,uit/ge/zwo/ren
99524,verzweren,ver/zwe/ren,NaN,NaN,ww.min_zich,NaN,verzwoor verzweerde,ver/zwoor ver/zweer/de,verzworen verzweerden,ver/zwo/ren ver/zweer/den,verzworen,ver/zwo/ren
110577,zweren1,zwe/ren,NaN,(een eed doen),ww.min_zich,NaN,zwoer,zwoer,NaN,NaN,gezworen,ge/zwo/ren
110578,zweren2,zwe/ren,NaN,(etteren),ww.min_zich,NaN,zwoor zweerde,zwoor zweer/de,zworen zweerden,zwo/ren zweer/den,gezworen,ge/zwo/ren


No idea how to fix this... **yet**. Leave it as is for now.

### Normalizing diacritics

TICCLAT has unicode wordforms, so we can just replace the diacritic markers with actual diacritics. How to do that?

Apparently, there is such a thing as "combining characters" in Unicode: https://stackoverflow.com/questions/34755556/how-do-i-add-accents-to-a-letter. Nice! Here's a table of them: https://en.wikipedia.org/wiki/Combining_character

In [23]:
# note that these are regex formatted, i.e. with special characters escaped
diacritic_markers = {'@`': '\u0300',    # accent grave
                     "@\\'": '\u0301',  # accent aigu
                     '@\\\\': '\u0308', # trema
                     '@\+': '\u0327',   # cedilla
                     '@\^': '\u0302',   # accent circumflex
                     '@=': '\u0303',    # tilde
                     '@@': "'",         # apostrophe (not actually a diacritic)
                     '@2': '\u2082',    # subscript 2
                     '@n': '\u0308n'    # trema followed by n
                    }

In [24]:
for marker, umarker in diacritic_markers.items():
    wordform_clean_df = wordform_clean_df.str.replace(marker, umarker)

In [25]:
wordform_clean_df.sort_values().head(10)

1               's anderendaags
2                 's-Gravenhage
3              's-Hertogenbosch
4     't is dief en diefjesmaat
5                      A-biljet
6                   A-biljetten
7                         A-bom
8                      A-bommen
9                      A-omroep
10                   A-omroepen
dtype: object

# Load Groene Boekje DataFrames into TICCLAT database

In [26]:
with ticclat.dbutils.session_scope(Session) as session:
    ticclat.dbutils.add_lexicon(session, "Groene Boekje 1995", wordform_clean_df.to_frame(name='wordform'))

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0308' in position 14: ordinal not in range(256)

Ok, this is giving me an error

```python-traceback
---------------------------------------------------------------------------
UnicodeEncodeError                        Traceback (most recent call last)
<ipython-input-93-22e27abe721c> in <module>
      1 with ticclat.dbutils.session_scope(Session) as session:
----> 2     ticclat.dbutils.add_lexicon(session, "Groene Boekje 1995", wordform_clean_df.to_frame(name='wordform'))

~/projects/ticclat/ticclat/ticclat/dbutils.py in add_lexicon(session, lexicon_name, wfs, num)
     97     in this case just "wordform"
     98     """
---> 99     bulk_add_wordforms(session, wfs, num=num)
    100 
    101     lexicon = Lexicon(lexicon_name=lexicon_name)

~/projects/ticclat/ticclat/ticclat/dbutils.py in bulk_add_wordforms(session, wfs, num)
     72 
     73         q = session.query(Wordform)
---> 74         result = q.filter(Wordform.wordform.in_(wordforms)).all()
     75 
     76         existing_wfs = [wf.wordform for wf in result]

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/orm/query.py in all(self)
   2923 
   2924         """
-> 2925         return list(self)
   2926 
   2927     @_generative(_no_clauseelement_condition)

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/orm/query.py in __iter__(self)
   3079         if self._autoflush and not self._populate_existing:
   3080             self.session._autoflush()
-> 3081         return self._execute_and_instances(context)
   3082 
   3083     def __str__(self):

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/orm/query.py in _execute_and_instances(self, querycontext)
   3104         )
   3105 
-> 3106         result = conn.execute(querycontext.statement, self._params)
   3107         return loading.instances(querycontext.query, result, querycontext)
   3108 

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/base.py in execute(self, object_, *multiparams, **params)
    978             raise exc.ObjectNotExecutableError(object_)
    979         else:
--> 980             return meth(self, multiparams, params)
    981 
    982     def _execute_function(self, func, multiparams, params):

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/sql/elements.py in _execute_on_connection(self, connection, multiparams, params)
    271     def _execute_on_connection(self, connection, multiparams, params):
    272         if self.supports_execution:
--> 273             return connection._execute_clauseelement(self, multiparams, params)
    274         else:
    275             raise exc.ObjectNotExecutableError(self)

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/base.py in _execute_clauseelement(self, elem, multiparams, params)
   1097             distilled_params,
   1098             compiled_sql,
-> 1099             distilled_params,
   1100         )
   1101         if self._has_events or self.engine._has_events:

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/base.py in _execute_context(self, dialect, constructor, statement, parameters, *args)
   1238         except BaseException as e:
   1239             self._handle_dbapi_exception(
-> 1240                 e, statement, parameters, cursor, context
   1241             )
   1242 

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/base.py in _handle_dbapi_exception(self, e, statement, parameters, cursor, context)
   1458                 util.raise_from_cause(sqlalchemy_exception, exc_info)
   1459             else:
-> 1460                 util.reraise(*exc_info)
   1461 
   1462         finally:

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/util/compat.py in reraise(tp, value, tb, cause)
    275         if value.__traceback__ is not tb:
    276             raise value.with_traceback(tb)
--> 277         raise value
    278 
    279 

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/base.py in _execute_context(self, dialect, constructor, statement, parameters, *args)
   1234                 if not evt_handled:
   1235                     self.dialect.do_execute(
-> 1236                         cursor, statement, parameters, context
   1237                     )
   1238         except BaseException as e:

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/default.py in do_execute(self, cursor, statement, parameters, context)
    534 
    535     def do_execute(self, cursor, statement, parameters, context=None):
--> 536         cursor.execute(statement, parameters)
    537 
    538     def do_execute_no_params(self, cursor, statement, context=None):

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/MySQLdb/cursors.py in execute(self, query, args)
    237                 args = dict((key, db.literal(item)) for key, item in args.items())
    238             else:
--> 239                 args = tuple(map(db.literal, args))
    240             if not PY2 and isinstance(query, (bytes, bytearray)):
    241                 query = query.decode(db.encoding)

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/MySQLdb/connections.py in literal(self, o)
    319             s = self._tuple_literal(o)
    320         else:
--> 321             s = self.escape(o, self.encoders)
    322         # Python 3(~3.4) doesn't support % operation for bytes object.
    323         # We should decode it before using %.

~/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/MySQLdb/connections.py in unicode_literal(u, dummy)
    227             # unicode_literal() is called for arbitrary object.
    228             def unicode_literal(u, dummy=None):
--> 229                 return db.string_literal(str(u).encode(db.encoding))
    230 
    231         def bytes_literal(obj, dummy=None):

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0308' in position 14: ordinal not in range(256)
```

It seems like the `db.encoding` for some reason is latin-1 there, even though we set the ticclat database to be utf8 in MySQL using

```mysql
CREATE DATABASE ticclat CHARACTER SET utf8mb4 COLLATE utf8mb4_bin;
```

This is a guide that should set everything to utf8: https://mathiasbynens.be/notes/mysql-utf8mb4#mysql-utf8mb4

The important part there that we're still missing is that `character_set_client` may still be non-utf8, i.e. clients could still read the data as latin-1, even though the database is encoded as utf8 (see e.g. https://nicj.net/mysql-converting-an-incorrect-latin1-column-to-utf8/). In my case it seems like its neither latin-1 nor utf8mb4, but actually "utf8" which is an alias for utf8mb3 (see https://stackoverflow.com/a/30074553/1199693), as I found out by running `SHOW VARIABLES LIKE 'character_set_client';`.

We will do this by creating a configuration file. To find out which file your client reads from, run this:
```sh
mysql --help | grep -A 1 "Default options are read from the following files"
```

For me this gives
```sh
Default options are read from the following files in the given order:
/etc/my.cnf /etc/mysql/my.cnf /Users/pbos/sw/miniconda3/envs/ticclat2/etc/my.cnf ~/.my.cnf
```

I'm using MySQL from Conda, so I'll use the miniconda location and put this in the file:

```ini
[client]
default-character-set = utf8mb4

[mysql]
default-character-set = utf8mb4

[mysqld]
character-set-client-handshake = FALSE
character-set-server = utf8mb4
collation-server = utf8mb4_unicode_ci
```

Then restart the server and recreate the database (don't care much about saving data at this point) by deleting and recreating.

```sh
mysqld restart
```

Indeed, now `SHOW VARIABLES LIKE 'character_set_client';` gives utf8mb4.

.......

Ok, never mind, I was using the "groene_boekje" database, not the "ticclat" database. D'oh!

.......

Ok, but with that fixed, the problem still persists. Even with the whole configuration file above added... crap.

In [28]:
with ticclat.dbutils.session_scope(Session) as session:
    ticclat.dbutils.add_lexicon(session, "Groene Boekje 1995", wordform_clean_df.str.encode('utf8').to_frame(name='wordform'))

/Users/pbos/sw/miniconda3/envs/ticclat2/lib/python3.7/site-packages/sqlalchemy/engine/default.py:536: Warning: (3170, '')
  cursor.execute(statement, parameters)



196934


Yaaay, that seems to have worked! The crucial addition there is `.str.encode('utf-8')`. Actually, we already discussed this, but I got lost in the MySQL settings and forgot.

This is also useful to check the encoding of tables themselves:
```mysql
SELECT T.table_name, CCSA.character_set_name FROM information_schema.`TABLES` T,
information_schema.`COLLATION_CHARACTER_SET_APPLICABILITY` CCSA
WHERE CCSA.collation_name = T.table_collation AND T.table_schema = "ticclat";
```

But also have to make sure that the data is put in correctly. Just browsing through the data in mysql with

```mysql
SELECT * FROM wordforms;
```

Shows that the unicode words are not displayed correctly in the terminal at least, e.g. `zooÌˆloog"` instead of `zoöloog`.

How about if we query from here?

In [64]:
'zoo{}loog'.format(diacritic_markers['@\\\\'])

'zoöloog'

In [72]:
# again gives long utf-8/latin-1 UnicodeEncodeError:
# with ticclat.dbutils.session_scope(Session) as session:
#     for wordform in session.query(ticclat.ticclat_schema.Wordform).filter_by(wordform='zoo{}loog'.format(diacritic_markers['@\\\\'])):
#          print(wordform)

# this gives no results at all
with ticclat.dbutils.session_scope(Session) as session:
    for wordform in session.query(ticclat.ticclat_schema.Wordform).filter_by(wordform='zooÌˆloog"'.encode('utf-8')):
         print(wordform)

# ... neither does this, when I manually type in zoöloog with alt-U O (on macOS) for the ö
with ticclat.dbutils.session_scope(Session) as session:
    for wordform in session.query(ticclat.ticclat_schema.Wordform).filter_by(wordform='zoöloog"'.encode('utf-8')):
         print(wordform)

# ... but when I copy-paste the output of the cell above it works!
with ticclat.dbutils.session_scope(Session) as session:
    for wordform in session.query(ticclat.ticclat_schema.Wordform).filter_by(wordform='zoöloog"'.encode('utf-8')):
         print(wordform)

<Wordform zooÌloog">
